In [1]:
!export VIRTUAL_ENV=/workspace/GreyLiterDocker/venv

In [3]:
%pipenv install

Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing dependencies from Pipfile.lock (aae065)...


In [2]:
!pipenv run pip list

Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
Package           Version
----------------- -----------
asttokens         2.4.1
comm              0.2.2
debugpy           1.8.2
decorator         5.1.1
exceptiongroup    1.2.2
executing         2.0.1
ipykernel         6.29.5
ipython           8.26.0
jedi              0.19.1
jupyter_client    8.6.2
jupyter_core      5.7.2
matplotlib-inline 0.1.7
nest-asyncio      1.6.0
packaging         24.1
parso             0.8.4
pexpect           4.9.0
pip               24.1
platformdirs      4.2.2
prompt_toolkit    3.0.47
psutil            6.0.0
ptyprocess        0.7.0
pure_eval         0.2.3
Pygments          2.18.0
python-dateutil   2.9.0.post0
pyzmq             26.0.

In [1]:
from tqdm.notebook import tqdm
import pandas as pd

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
DEV = False
# model_name = 'dunzhang/stella_en_400M_v5'
model_name = "avsolatorio/GIST-Embedding-v0" # train when I've got a spare two hours

: 

In [ ]:
def import_labelled_data(path="data/labelled/data.json", group_relevant=True):
    data = pd.read_json(path, encoding="latin-1")
    data["relevance"] = data["class"].apply(
        lambda x: "relevant" if x != "irrelevant" else x
    )
    return data

data = import_labelled_data(path='../../data/labelled/data.json',group_relevant=False)

# drop null classes
data = data.dropna(subset=["class"])

if DEV:
    data = data.sample(5000)

# train test split
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

data.head()

: 

In [ ]:
from chunking import chunk_dataset_and_explode


# roughly 4 characters per token
max_len = 2048

train_data = chunk_dataset_and_explode(train_data, max_len=max_len, overlap=int(max_len * 0.2))
test_data = chunk_dataset_and_explode(test_data, max_len=max_len, overlap=int(max_len * 0.2))

: 

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_data, split="train")
test_dataset = Dataset.from_pandas(test_data, split="test")

train_dataset

: 

In [ ]:
# embeddings
from sentence_transformers import SentenceTransformer


model = SentenceTransformer(model_name,trust_remote_code=True,similarity_fn_name='dot')

model.cuda()
# model.cpu()

: 

In [ ]:
model.device

: 

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))
# get version of cuda
print(torch.version.cuda)
# clear cache
torch.cuda.empty_cache()

: 

In [ ]:
train_embeddings = model.encode(
    train_dataset["text"],
    show_progress_bar=True,
    normalize_embeddings=True,
)

: 

In [ ]:
test_embeddings = model.encode(
    test_dataset["text"],
    show_progress_bar=True,
    normalize_embeddings=True,
)

: 

In [ ]:
train_embeddings.shape

: 

In [ ]:
train_embeddings[0]

: 

In [ ]:
# add to pandas dataframe
train_data["embeddings"] = train_embeddings.tolist()
test_data["embeddings"] = test_embeddings.tolist()

: 

In [ ]:
train_data.head()

: 

In [ ]:
import os

# Create the folder if it doesn't exist
if not os.path.exists(f"embeddings/{model_name}/dev"):
    os.makedirs(f"embeddings/{model_name}/dev")

if DEV:
    train_data.to_json(f"embeddings/{model_name}/dev/train_embeddings.json", orient='records', indent=4)
    test_data.to_json(f"embeddings/{model_name}/dev/test_embeddings.json", orient='records', indent=4)
else:
    train_data.to_json(f"embeddings/{model_name}/train_embeddings.json", orient='records')
    test_data.to_json(f"embeddings/{model_name}/test_embeddings.json", orient='records')

    # also save dev data
    devTrain = train_data.sample(10000)
    devTest = test_data.sample(int(10000*0.2))
    devTrain.to_json(f"embeddings/{model_name}/dev/train_embeddings.json", orient='records')
    devTest.to_json(f"embeddings/{model_name}/dev/test_embeddings.json", orient='records')



: 